## OpenStreetMap Valhalla API



### このノートブックの流れ

- 生成 AI に、自然言語から出発地と目的地の地名のみを出力させる
- 出力された出発地と目的地の地名に基づいて Nominatim API を呼び出して GPS 座標を取得する
- 取得した GPS 座標のペアに基づいて Valhalla API で経路情報を取得する
- 取得した経路情報を地図上に表示する

### 前準備

### 対象となる自然言語

In [1]:
input_text = "長崎空港から出島メッセ長崎に行きたいです。"
print(input_text)

長崎空港から出島メッセ長崎に行きたいです。


### 生成 AI に自然言語から出発地と目的地の地名のみを出力させる

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# モデルの準備
model = ChatGoogleGenerativeAI(model="gemini-exp-1206", temperature=0)

# プロンプトの準備
template = """You are a named entity recognition model.
Extract the location name of the departure and destination from the following input text.

You must follow the following format:
- The departure location name must be extracted first.
- The destination location name must be extracted second.
- Extract and output only the location name.

Input:
{input}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

res = chain.invoke({"input": input_text})
result = res.content.strip()
print(result)

- 長崎空港
- 出島メッセ長崎


In [3]:
departure_text, destination_text = result.split("\n")
departure_text = departure_text.split("- ")[1]
destination_text = destination_text.split("- ")[1]
print(departure_text)
print(destination_text)

長崎空港
出島メッセ長崎


### Nominatim API で出発地と目的地の GPS 座標を取得する

In [4]:
import requests

def get_nominatim_response(query: str):
    params = {
        "format": "jsonv2",
        "polygon_geojson": "0",
        "q": query
    }
    headers = {
        "User-Agent": "NLP2025/1.0"
    }
    nominatim_url = "https://nominatim.openstreetmap.org/search"
    response = requests.get(nominatim_url, params=params, headers=headers)
    if response.status_code == 200:
        res_json = response.json()
        return res_json
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [5]:
import time
departure_res = get_nominatim_response(departure_text)
departure_lat = departure_res[0]["lat"]
departure_lon = departure_res[0]["lon"]
departure = {"lat": departure_lat, "lon": departure_lon}
print(departure)
time.sleep(1)
destination_res = get_nominatim_response(destination_text)
destination_lat = destination_res[0]["lat"]
destination_lon = destination_res[0]["lon"]
destination = {"lat": destination_lat, "lon": destination_lon}
print(destination)

{'lat': '32.91685555', 'lon': '129.91653868964397'}
{'lat': '32.752506600000004', 'lon': '129.86764840717382'}


### Valhalla API で出発地から目的地までの経路情報を取得する

In [6]:
import json

def get_valhalla_response(departure, destination):
    if not all(k in departure for k in ('lat', 'lon')) or not all(k in destination for k in ('lat', 'lon')):
        raise ValueError("Both departure and destination must be dictionaries with 'lat' and 'lon' keys.")
    payload = {
        "costing": "pedestrian",
        "costing_options": {
            "pedestrian": {
                "exclude_polygons": [],
                "use_ferry": 1,
                "use_living_streets": 0.5,
                "use_tracks": 0,
                "service_penalty": 15,
                "service_factor": 1,
                "shortest": False,
                "use_hills": 0.5,
                "walking_speed": 5.1,
                "walkway_factor": 1,
                "sidewalk_factor": 1,
                "alley_factor": 2,
                "driveway_factor": 5,
                "step_penalty": 0,
                "max_hiking_difficulty": 1,
                "use_lit": 0,
                "transit_start_end_max_distance": 2145,
                "transit_transfer_max_distance": 800,
            }
        },
        "exclude_polygons": [],
        "locations": [departure, destination],
        "directions_options": {"units": "kilometers"},
        "id": "valhalla_directions",
    }
    json_payload = json.dumps(payload, separators=(",", ":"))
    params = {"json": json_payload}
    valhalla_url = "https://valhalla1.openstreetmap.de/route"
    response = requests.get(valhalla_url, params=params)
    if response.status_code == 200:
        res_json = response.json()
        return res_json
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [7]:
valhalla_response = get_valhalla_response(departure, destination)
print(json.dumps(valhalla_response, indent=2, ensure_ascii=False))

{
  "trip": {
    "locations": [
      {
        "type": "break",
        "lat": 32.916855,
        "lon": 129.916538,
        "original_index": 0
      },
      {
        "type": "break",
        "lat": 32.752506,
        "lon": 129.867648,
        "side_of_street": "right",
        "original_index": 1
      }
    ],
    "legs": [
      {
        "maneuvers": [
          {
            "type": 1,
            "instruction": "Walk southwest.",
            "verbal_succinct_transition_instruction": "Walk southwest.",
            "verbal_pre_transition_instruction": "Walk southwest.",
            "verbal_post_transition_instruction": "Continue for 100 meters.",
            "bearing_after": 228,
            "time": 81.176,
            "length": 0.115,
            "cost": 81.176,
            "begin_shape_index": 0,
            "end_shape_index": 1,
            "travel_mode": "pedestrian",
            "travel_type": "foot"
          },
          {
            "type": 15,
            "instructi

In [8]:
print(valhalla_response['trip']['legs'][0]['shape'])

qr~w}@osjxvFfj@xw@hxDgdEtwDudE|EwEzH_IfyC{_BbCoA`BgB`AiBv@uBnXysAj@kD?oC[uCy@}ByvAisBck@}x@mA{AeBwAgBm@oBMgBFmBd@mBbAyAzAydDnpD{T~UiBzAkBn@eBTsB?aB[eBo@wA_Bol@a{@oVgY_AyAq@uAx@i@t@Wr@Bb@NeEeHyZqd@gIcMc@q@s{IwqN}l@{`AoZ}i@yVch@oIcQ{`@my@}CqLdAcA}@iEo@}C}@oGYiJi@qWKiGt@}BzAuB~XoXxUmVxB}BpEsFde@wf@`RsRhk@yk@_DkDvi@al@td@ol@zEwGxe@gp@nNyRlLaOzRgX|EyGvOgT|]if@dHuJxLsPhE_GhBgCn_@kh@dp@s}@bIoM`FkL~@_DfAaEfDmOfDeOfEiR~Mis@dAqFzE{`@b]y^nNoOr@s@~_@w`@`MyMtKwLlNeOrHcIdEqEvDaEzKaMh@k@x_@ub@tGmHnAsAdOmKbTgPjA}@`IgJhDsCrZo]|DaErL}LlMyJnFcEjHkFfOuIjSsHlMmCvUoGhUoEvSeE`VuF|JaCbWmFjNqCt[mGt]eH|q@mNi@qEpn@qMrUmFkBmMuHmf@eAkHwIyk@z`@cJpBo@tWmGtQkFvJeEnk@qXaCoFs@}A_CmFoBiE}EaLc@_AlNaJJYdGmDdIuE`Aq@TMrJqFvBmBjD}Cx@sEQsDkDcNKcEdDgEjCu@x^oQjDuEvRmTnByBxCkDl@s@jBqAx@k@j`AmqAcB}AmD_FbI}MpFiJ`CwEtD}FvGyFvJ}BhF]`@qMfAeNpBcMvBgJ|HaTrDeH|IwKdKyKjBvDxHsF|KaFzIgD|UmG~FsArBe@fRiEji@eJrTuErM_DbJqC|MoE`BwAvHwGjHiI~EsI`E_JnEgNdC}N~AcObA_YSuP]qK}@qL_BcLiCqNaFcPyFiRsFcQa[e`AgHgK}IkXuFy[_@gCcB_MeEec@q@gQSwQ^cLpAqP~B}NtFyWn

### Valhalla API で取得したデータを GeoJSON に変換する

In [9]:
def decode_valhalla_polyline(encoded: str, mul: float):
    # Precision multiplier inverse
    inv = 1.0 / mul
    decoded = []
    previous = [0, 0]
    i = 0
    while i < len(encoded):
        ll = [0, 0]
        for j in range(2):
            shift = 0
            byte = 0x20
            while byte >= 0x20:
                byte = ord(encoded[i]) - 63
                i += 1
                ll[j] |= (byte & 0x1f) << shift
                shift += 5
            # Decode and add the previous offset
            ll[j] = previous[j] + (ll[j] & 1 and ~(ll[j] >> 1) or (ll[j] >> 1))
            previous[j] = ll[j]
        # Scale by precision and flip positions to lon, lat
        decoded.append([ll[1] * inv, ll[0] * inv])
    return decoded

In [10]:
def convert_valhalla_to_geojson(response):
    decoded = decode_valhalla_polyline(response['trip']['legs'][0]['shape'], 1e6)
    geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": decoded
                },
                "properties": {}
            }
        ]
    }
    return geojson

In [11]:
geojson_data = convert_valhalla_to_geojson(valhalla_response)
print(json.dumps(geojson_data, indent=2, ensure_ascii=False))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            129.914696,
            32.915257
          ],
          [
            129.91378699999999,
            32.914564999999996
          ],
          [
            129.916943,
            32.9116
          ],
          [
            129.920106,
            32.908645
          ],
          [
            129.920214,
            32.908533999999996
          ],
          [
            129.92037399999998,
            32.908376
          ],
          [
            129.921924,
            32.905908
          ],
          [
            129.921964,
            32.905842
          ],
          [
            129.92201599999999,
            32.905792999999996
          ],
          [
            129.922069,
            32.90576
          ],
          [
            129.922128,
            32.905732
          ],
          [
    

### GeoJSON を地図上に表示する

In [12]:
import folium

# default map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the GeoJSON data to the map
folium.GeoJson(geojson_data).add_to(m)

# Get bounds from GeoJSON data and fit the map to those bounds
# Extract bounds from geojson_data
bounds = folium.GeoJson(geojson_data).get_bounds()
# Ensure bounds are valid before fitting
if bounds:
    m.fit_bounds(bounds)

# Display the map
display(m)